In [1]:
# allow for imports from ../src directory

from os import path as os_path, getcwd
from sys import path as sys_path

sys_path.append(os_path.dirname(getcwd())) 
sys.path.append(f"{os_path.dirname(getcwd())}/src")

In [2]:
print(sys.path)

['/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/notebooks', '/Users/mjboothaus/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles', '/Users/mjboothaus/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles/lib/python', '/Users/mjboothaus/.pyenv/versions/3.9.12/lib/python39.zip', '/Users/mjboothaus/.pyenv/versions/3.9.12/lib/python3.9', '/Users/mjboothaus/.pyenv/versions/3.9.12/lib/python3.9/lib-dynload', '', '/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/.venv_dev_michael-booth-phd-thesis/lib/python3.9/site-packages', '/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis', '/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/src']


In [5]:
from dataclasses import dataclass

import scipy.optimize as optim
import numpy as np
import pandas as pd
import siunits as unit

from scipy import interpolate
from scipy.integrate import trapezoid
from scipy.constants import epsilon_0, elementary_charge, Boltzmann, Avogadro
from src.helper_functions import read_render_markdown_file
from src.plotting import make_simple_plot, fast_plot, plotly_line

from src.modelling import *
from src.numerics import set_num_parameters

ModuleNotFoundError: No module named 'siunits'

In [9]:
tw = np.array([0, 1, 2, 3, 4])

z = np.linspace(0, 50, 51)

In [10]:
z

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.])

In [8]:
tw[1:], tw[:-1]

(array([1, 2, 3, 4]), array([0, 1, 2, 3]))

In [11]:
z[1:] - z[:-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Physical constants

1. the electric constant (vacuum permittivity) $\epsilon_0$. Units:  $\textstyle{\rm s^{4}\cdot A^{2}\cdot kg^{-1}\cdot m^{-3}}$ or $\textstyle{\rm C^2 \cdot N^{−1} \cdot m^{−2}}$ or $\textstyle{\rm C \cdot V^{−1} \cdot m^{−1}}$ 

In [ ]:
unit.epsilon_0

In [ ]:
epsilon_0

2. Elementary charge (charge on electron). Units: $\rm C$

In [ ]:
unit.c

In [ ]:
elementary_charge

3. Boltzmann constant. Units: $\rm m^2 \cdot kg \cdot s^{-2} \cdot {K}^{-1}$

In [ ]:
Boltzmann

4. Avogadro's number. Units: $\rm mol^{-1}$

In [ ]:
Avogadro

### Singlet model parameters

See Page 62

Dictionary of model parameters

In [ ]:
fluid = "kcl"

In [ ]:
fluid = set_fluid_parameters("kcl")

In [ ]:
fluid

In [ ]:
fluid.rho = np.array([1, 1, 1])

In [ ]:
fluid.rho

In [ ]:
valence = fluid.valence
concentration = fluid.concentration
n_pair = fluid.n_pair
n_component = fluid.n_component

Numerical discretisation

In [ ]:
z_cutoff = 50.0    # from UI
n_point = 2001     # from UI

In [ ]:
tolerance = 1e-9
max_iteration = 100

In [ ]:
discrete = set_num_parameters(n_point, z_cutoff, n_component, n_pair, tolerance, max_iteration)

In [ ]:
grid_size = discrete.grid_size
z = np.linspace(0.0, z_cutoff, n_point)
z_index = np.arange(0, n_point, dtype=int)

Calculate physical constants

$\beta = 1 / k T$. Units: $\rm m^{-2} \cdot kg^{-1} \cdot s^{2}$

In [ ]:
beta = 1.0 / (Boltzmann * fluid.temperature)

In [ ]:
epsilon = 4.0 * np.pi * fluid.epsilon_r * epsilon_0  # units same as $\epsilon_0$ (allow for distances in angstrom)

Model potential KCl

In [ ]:
read_render_markdown_file("../docs/equation_4_1.md")
read_render_markdown_file("../docs/equation_4_2.md")
read_render_markdown_file("../docs/equation_4_3.md")

In [ ]:
n_outer_shell = np.array([8., 8.])

In [ ]:
# def calc_l_index(i, j):
#     return i + j    

In [ ]:
# def calc_beta_pauling(valence, n_outer_shell):
#     beta_pauling = np.zeros(n_pair)
#     for i in range(n_component):
#         for j in range(i, n_component):
#             l = calc_l_index(i, j)
#             beta_pauling[l] = 1.0 + valence[i] / n_outer_shell[i] + valence[j] / n_outer_shell[j]
#     return beta_pauling

In [ ]:
beta_pauling = calc_beta_pauling(fluid.valence, n_outer_shell, n_component, n_pair)

In [ ]:
beta_pauling

In [ ]:
alpha = 1.0 / 0.337
b = 0.338e-19
sigma = [1.463, 1.585]

cap_c = np.array([24.3, 48.0, 124.5]) * 1e-19
cap_d = np.array([24.0, 73.0, 250.0]) * 1e-19

In [ ]:
def calc_cap_b(beta_pauling, b, alpha, sigma):
    cap_b = np.zeros(n_pair)
    for i in range(n_component):
        for j in range(i, n_component):
            l = calc_l_index(i, j)
            cap_b[l] = beta_pauling[l] * b * np.exp(alpha * (sigma[i] + sigma[j]))
    return cap_b

In [ ]:
cap_b = calc_cap_b(beta_pauling, b, alpha, sigma)

In [ ]:
cap_b

In [ ]:
#TODO: Fix up choice of c(r) / c*(r) and make sure discretisation matches
# Read in some c(r) -- currently LJ charged spheres

def load_cr():
    CR_PATH = "../pyOZ_bulk_fluid/tests/lj/nrcg-cr.dat.orig"
    cr_df = pd.read_csv(CR_PATH, header=None, delim_whitespace=True)
    cr_df.set_index(0, inplace=True)
    r = cr_df.index.to_numpy()
    cr = cr_df.to_numpy()
    return cr, r

In [ ]:
cr_in, r_in = load_cr()

In [ ]:
cr_in.shape, r_in.shape

In [ ]:
type(r_in)

In [ ]:
make_simple_plot(r_in, cr_in, "r", "cr", "cr", yliml=[-2, 2])

In [ ]:

def interpolate_cr(r_in, cr_in):
    cr = np.zeros((n_point, n_pair))
    for l in range(n_pair):
        f = interpolate.interp1d(r_in, cr_in[:, l])
        r = z
        cr[:, l] = f(r)
    return cr, r

In [ ]:
cr, r = interpolate_cr(r_in, cr_in)

In [ ]:
make_simple_plot(r, cr, "r", "cr", "cr", yliml=[-2, 2])

In [ ]:
def calc_charge(valence):
    return valence * elementary_charge

In [ ]:
charge = calc_charge(valence)

In [ ]:
r_m = r * 1e-10  # convert distance from Angstrom to m

In [ ]:
# Use same discretisation for r as z [Confirm?]

# def calc_u(valence, cap_b, alpha, cap_c, cap_d, r):
#     u = np.zeros((n_point, n_pair))
#     for i in range(n_component):
#         for j in range(i, n_component):
#             l = calc_l_index(i, j)
#             u[1:, l] = (charge[i] * charge[j]) / (r_m[1:] * epsilon) + cap_c[l] / \
#                 r[1:]**6 + cap_d[l]/r[1:]**8 + \
#                 cap_b[l] * np.exp(-alpha * r[1:])
#     return u


In [ ]:
#TODO: Sort out units for this - doesn't make sense currently

u = calc_u(charge, cap_b, alpha, cap_c, cap_d, n_point, n_component, n_pair, epsilon, r)

In [ ]:
make_simple_plot(r, beta * u, "r", "u", "Dimensionless ion-ion potential", xliml=[0, 10], yliml=[-100, 200])

In [ ]:
fig = plotly_line(r, beta*u, ["r", "u0", "u1", "u2"], xliml=[0, 10], yliml=[-100, 200])
fig.show()

In [ ]:
# f.plot(x="r", y=["u0", "u1", "u2"], xlim=[0, 10], ylim=[-100, 200])

In [ ]:
psi_0 = 100. / 1000.     # 100 mV (in Volts)  -- get from UI

rho ($\rho$) is the number density which we calculate from the concentration ($c$):

$\rho = c \, \rm mol \, dm^{-3} \cdot dm^{3} / 1.0 \cdot 10^{27} \cdot A^{3} \cdot N_{A} \, mol^{-1}$. Units: $\rm A^{-3}$


In [ ]:
def calc_rho(concentration):
    return np.array(concentration) / 1.0e27 * Avogadro

In [ ]:
rho = calc_rho(concentration)

In [ ]:
rho   # values seem reasonable (compared with those in thesis)

In [ ]:
charge = calc_charge(valence)

In [ ]:
def calc_kappa(beta, rho, epsilon):
    return np.sqrt(4.0 * np.pi * beta / epsilon * 1e10 *
    sum(np.multiply(charge**2, rho)))


In [ ]:
kappa = calc_kappa(beta, rho, epsilon)

In [ ]:
kappa      # similar to thesis value (6.77) page 62 - maybe revisions to physical constants in past 25 years (e.g. 2019 change for $e$)

### Wall-ion soft-potential

In [ ]:
phiw = np.zeros((n_point, n_component))
hw = np.zeros((n_point, n_component))
tw = np.zeros((n_point, n_component))
# integral_0_z = np.zeros((n_point, n_component))
# integral_z_inf = np.zeros((n_point, n_component))

In [ ]:
# same for both ions

def calc_phiw(z, n_component):
    capital_a = 16.274e-19 # joules
    wall_d = 2.97  # inverse Angstrom
    for i in range(n_component):
        phiw[:, i] = np.exp(-wall_d * z) * capital_a * (wall_d * z + 2)
    return phiw


In [ ]:
beta_phiw = beta * calc_phiw(z, n_component)

In [ ]:
make_simple_plot(z, beta_phiw, "z", "$beta phi_{wi}(z)$", "Dimensionless wall-ion potentials")

This dimensionless plot matches well with Figure 4.1(b) of thesis.

### Bulk fluid inputs

Not sure what to do here -- maybe run pyOZ for ionic fluid and try using c_{ij}(r) from this as an approximation?

In [ ]:
read_render_markdown_file("../docs/equation_3_4.md")

In [ ]:
# Assume all of the moments of the short-ranged bulk direct correlation functions to be zero initially

f1 = np.zeros((n_point, n_pair))
f2 = np.zeros((n_point, n_pair))
c_short = np.zeros((n_point, n_pair))

In [ ]:
# for now assume c*(r) = c(r)

c_short = cr

In [ ]:
charge_pair = np.array([1, -1, 1]) * elementary_charge

In [ ]:
read_render_markdown_file("../docs/equation_3_5.md")

In [ ]:
#TODO: Too big?

beta * charge_pair[0] * elementary_charge * elementary_charge / (epsilon * 1e-10) 

In [ ]:
c_short

In [ ]:
# skip r = 0 to avoid divide by 0 error

#for l in range(n_pair):
#    c_short[1:, l] = c_short[1:, l]  + beta * charge_pair[l] * 1e-10 / (epsilon * r[1:])
    

In [ ]:
c_short

In [ ]:
make_simple_plot(r, c_short, "r", "$c_{ij}^*$", "SR direct correlation ion-ion functions", xliml=[0, 20], yliml=[-2, 2])

In [ ]:
def integral_z_infty_dr_r_c_short(c_short, z):
    for ij in range(n_pair):
        for k, _ in enumerate(z):
            f1[k, ij] = trapezoid(y=z[:k] * c_short[k, ij], x=z[:k])
    return f1

In [ ]:
def integral_z_infty_dr_r2_c_short(c_short, z):
    for ij in range(n_pair):
        for k, _ in enumerate(z):
            f2[k, ij] = trapezoid(y=z[:k] * z[:k] * c_short[k, ij], x=z[:k])
    return f2

In [ ]:
f1 = integral_z_infty_dr_r_c_short(c_short, z)

In [ ]:
make_simple_plot(z, f1, "z", "$f_{ij}(z)$", "Moment 1 of c^*(r)", yliml=[-15, 5])

In [ ]:
f2 = integral_z_infty_dr_r2_c_short(c_short, z)

In [ ]:
make_simple_plot(z, f2, "z", "$f2_{ij}(z)$", "Moment 2 of c^*(r)", yliml=[-40, 5])

Initial guess for solution $t_{wi}(z)$ - solving $F(t(z)) = 0$

In [ ]:
tw_initial = np.zeros((n_point, n_component))    # initial guess of zero - maybe should be \beta \phi

# tw = beta_phiw_i

In [ ]:
make_simple_plot(z, tw_initial, "z", "$t_{wi}(z)$", "Initial guess for $t_{wi}(z)$")

In [ ]:
def calc_hw(tw, beta_phiw):
    for i in range(n_component):
        hw[:, i] = np.exp(tw[:, i] - beta_phiw[:, i]) - 1.0
    return hw


In [ ]:
hw = calc_hw(tw_initial, beta_phiw)

In [ ]:
make_simple_plot(z, hw, "z", "$h_{wi}(z)$", "Initial version of $h_{wi}(z)$", yliml=[-1, 2])

In [ ]:
beta_psi =  beta * psi_0

In [ ]:
read_render_markdown_file("../docs/equation_5_9.md")
read_render_markdown_file("../docs/equation_5_9b.md")

In [ ]:
integral_0_z = np.zeros((n_point, n_component))
integral_z_infty = np.zeros((n_point, n_component))

In [ ]:
beta_psi * charge[0]

In [ ]:
2.0 * beta * charge[0] * charge[0] / epsilon * 1e10

#### TODO: Check limits to make sure treating end points of arrays appropriately



In [ ]:
beta_psi_charge = np.zeros((n_point, n_component))

In [ ]:
def calc_tw_term_by_term(tw_in, beta_psi, rho, f1, f2, z):
    hw = calc_hw(tw_in, beta_phiw)

    for i in range(n_component):
        for k in range(n_point):
            integral_0_z[k, i] = trapezoid(y=hw[:k, i], x=z[:k])
            integral_z_infty[k, i] = trapezoid(y=z[k:] * hw[k:, i], x=z[k:])

    for i in range(n_component):
        beta_psi_charge[:, i] = -beta_psi * charge[i]
        for k in range(n_point):
            z_minus_t = np.flip(z_index[:k])
            t_minus_z = z_index[k:] - k
            for j in range(n_component):
                l = calc_l_index(i, j)
                tw[k, i] = beta_psi_charge[k, i] + 2.0 * np.pi * rho[j] * (z[k] * f1[k, l] - f2[k, l])
                # +
                #                                               (2.0 * beta * charge[i] * charge[j] / epsilon) *
                #                                               (integral_z_infty[k, j] + z[k] * integral_0_z[k, j])
                #                                               + trapezoid(y=hw[:k, j] * f1[z_minus_t, l])
                #                                               + trapezoid(y=hw[k:, j] * f1[t_minus_z, l]))
    return hw, integral_0_z, integral_z_infty, beta_psi_charge, z_minus_t, t_minus_z, tw, k

In [ ]:
hw_initial, integral_0_z, integral_z_infty, beta_psi_charge, z_minus_t, t_minus_z, tw, k = calc_tw_term_by_term(tw_initial, beta_psi, rho, f1, f2, z)

In [ ]:
make_simple_plot(z, hw_initial, title="hw_initial", yliml=[-1, 0])
make_simple_plot(z, integral_0_z, title="integral_0_z", yliml=[-5, 5])
make_simple_plot(z, integral_z_infty, title="integral_z_infty", yliml=[-5, 5])
make_simple_plot(z, beta_psi_charge, title="beta_psi_charge")
make_simple_plot(z, tw, title="tw", yliml=[-10, 10])
make_simple_plot(z, calc_hw(tw, beta_phiw), title="hw", yliml=[-2, 8])

In [ ]:
def calc_tw(tw_in, beta_psi, rho, f1, f2, z):
    hw = calc_hw(tw_in, beta_phiw)

    for i in range(n_component):
        for k in range(n_point):
            integral_0_z[k, i] = trapezoid(y=hw[:k, i], x=z[:k])
            integral_z_infty[k, i] = trapezoid(y=z[k:] * hw[k:, i], x=z[k:])

    for i in range(n_component):
        beta_psi_charge[:, i] = -beta_psi * charge[i]
        for k in range(n_point):
            z_minus_t = np.flip(z_index[:k])
            t_minus_z = z_index[k:] - k
            for j in range(n_component):
                l = calc_l_index(i, j)
                tw[k, i] = beta_psi_charge[i]
                tw[k, i] += TWO_PI * rho[j] * (z[k] * f1[k, l] - f2[k, l]
                                                + charge_pair[l] * (integral_z_infty[k, j] + z[k] * integral_0_z[k, j])
                                                + trapezoid(y=hw[:k, j] * f1[z_minus_t, l])
                                                + trapezoid(y=hw[k:, j] * f1[t_minus_z, l]))
    return tw


### Numerical solver: Newton-GMRES

In [ ]:
# this should equal 0 (as t(z) converges to solution)

def opt_func(tw):
    return tw - calc_tw(tw, beta_psi, rho, f1, f2, z)

In [ ]:
# Documentation: https://scipy.github.io/devdocs/reference/optimize.root-krylov.html

solution = optim.root(opt_func, tw_initial, method="krylov", jac=None,
                      tol=tolerance, callback=None, options={"disp": True, "maxiter": max_iteration})


In [ ]:
tw_solution = solution.x

In [ ]:
solution.nit

In [ ]:
make_simple_plot(z, tw_solution, "z", "t_{wi}(z)$", "Solution for $t_{wi}(z)$", xliml=[0, 20], yliml=[-1, 10])

In [ ]:
hw_solution = calc_hw(tw_solution, beta_phiw)

In [ ]:
make_simple_plot(z, hw_solution+1, "z", "g_{wi}(z)$", "Solution for $g_{wi}(z)$", xliml=[0, 20], yliml=[0, 10])

In [ ]:
assert np.sum(opt_func(tw)) < tolerance